<a href="https://colab.research.google.com/github/Dhanush-adk/intro_to_dl/blob/main/Assignment_5/Assignment_5_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.3 MB/s eta 0:00:00
time: 365 µs (started: 2024-04-26 22:16:28 +00:00)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
import numpy as np
import requests

time: 7.26 s (started: 2024-04-26 22:16:28 +00:00)


for sequence 30

In [3]:
# Step 1: Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # This is the entire text data

# Step 2: Prepare the dataset
sequence_length = 30
# Create a character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode the text into integers
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert lists to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Step 3: Create a dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Instantiate the dataset
dataset = CharDataset(sequences, targets)

# Step 4: Create data loaders
batch_size = 128
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(test_dataset, shuffle = False, batch_size = batch_size)


time: 11.4 s (started: 2024-04-26 22:16:36 +00:00)


In [4]:
class CharTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead):
        super(CharTransformer, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output[:, -1, :])  # Get the output of the last Transformer block
        return output


time: 1.05 ms (started: 2024-04-26 22:16:47 +00:00)


In [5]:
hidden_size = 512
num_layers = 2
num_heads = 2
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 15

time: 50.9 ms (started: 2024-04-26 22:16:47 +00:00)


In [6]:
model = CharTransformer(len(chars), hidden_size, len(chars), num_layers, num_heads).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # Move data to device
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

    loss = train_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to device
            val_output = model(inputs)
            loss = criterion(val_output, targets)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(val_output, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    loss_val = val_loss / len(test_loader.dataset)
    val_accuracy = correct / total

    if (epoch+1) % 1 == 0:
        print(f'Epoch {epoch+1}, loss: {loss}, val loss: {loss_val}, val accuracy: {val_accuracy}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1, loss: 2.854694128036499, val loss: 2.843194618173924, val accuracy: 0.1950661890950496
Epoch 2, loss: 2.7159008979797363, val loss: 2.7531382579344057, val accuracy: 0.20747020033800595
Epoch 3, loss: 2.8177478313446045, val loss: 2.750781629827849, val accuracy: 0.23314789329053717
Epoch 4, loss: 2.783623218536377, val loss: 2.7404696324800466, val accuracy: 0.23060612445253348
Epoch 5, loss: 2.666354179382324, val loss: 2.6632753026468183, val accuracy: 0.24521120888677697
Epoch 6, loss: 2.627310276031494, val loss: 2.639702974567084, val accuracy: 0.23513827312135488
Epoch 7, loss: 2.742638111114502, val loss: 2.6830802686950643, val accuracy: 0.24001111743689285
Epoch 8, loss: 2.7394111156463623, val loss: 2.704764816725779, val accuracy: 0.2411587238258328
Epoch 9, loss: 2.664944887161255, val loss: 2.640737367484444, val accuracy: 0.2457043210070246
Epoch 10, loss: 2.688864231109619, val loss: 2.648195458680766, val accuracy: 0.24683847888359417
Epoch 11, loss: 2.6763570

In [7]:
!pip install torchinfo
import torchinfo

time: 5.03 s (started: 2024-04-26 23:46:13 +00:00)


In [8]:
dataiter = iter(train_loader)
inputs, labels = next(dataiter)  # Get one batch of data

model.to(inputs.device)
summary = torchinfo.summary(model, input_data=(inputs,))
print(summary)


Layer (type:depth-idx)                        Output Shape              Param #
CharTransformer                               [128, 65]                 --
├─Embedding: 1-1                              [128, 30, 512]            33,280
├─TransformerEncoder: 1-2                     [128, 30, 512]            --
│    └─ModuleList: 2-1                        --                        --
│    │    └─TransformerEncoderLayer: 3-1      [128, 30, 512]            3,152,384
│    │    └─TransformerEncoderLayer: 3-2      [128, 30, 512]            3,152,384
├─Linear: 1-3                                 [128, 65]                 33,345
Total params: 6,371,393
Trainable params: 6,371,393
Non-trainable params: 0
Total mult-adds (M): 546.58
Input size (MB): 0.03
Forward/backward pass size (MB): 236.00
Params size (MB): 17.08
Estimated Total Size (MB): 253.11
time: 1.01 s (started: 2024-04-26 23:46:18 +00:00)


time: 1.02 s (started: 2024-04-26 23:46:18 +00:00)
